In [34]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [95]:
X_train = pd.read_csv('data/train_kan.csv')
X_test = pd.read_csv('data/test_kan.csv')

In [57]:
X = torch.tensor(X_train.iloc[:, 1:].values)
y = torch.tensor(X_train.iloc[:, :1].values)
y.shape

torch.Size([42000, 1])

In [58]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 200)
        self.l2 = nn.Linear(200, 10)
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(F.relu(x))
        return x

In [73]:
myNet = Net()
optimizer = torch.optim.AdamW(myNet.parameters(), lr=0.01)
epochs = 2000
# decay_rate = 0.2
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

for epoch in range(epochs+1):
    optimizer.zero_grad()
    bs = torch.randint(0, X.shape[0], (64,))
    logits = myNet(X[bs].float())
    loss = F.cross_entropy(logits, y[bs].squeeze())
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} / {epochs} Loss: {loss.item()}")


Epoch: 0 / 2000 Loss: 50.04924774169922
Epoch: 100 / 2000 Loss: 1.1326535940170288
Epoch: 200 / 2000 Loss: 1.059799313545227
Epoch: 300 / 2000 Loss: 0.8913412094116211
Epoch: 400 / 2000 Loss: 0.9665771126747131
Epoch: 500 / 2000 Loss: 0.7066649794578552
Epoch: 600 / 2000 Loss: 0.7968693971633911
Epoch: 700 / 2000 Loss: 0.8023806214332581
Epoch: 800 / 2000 Loss: 0.5425399541854858
Epoch: 900 / 2000 Loss: 0.6825492978096008
Epoch: 1000 / 2000 Loss: 0.9766929745674133
Epoch: 1100 / 2000 Loss: 0.7693346738815308
Epoch: 1200 / 2000 Loss: 0.822264552116394
Epoch: 1300 / 2000 Loss: 0.8588532209396362
Epoch: 1400 / 2000 Loss: 1.083083987236023
Epoch: 1500 / 2000 Loss: 0.7164874076843262
Epoch: 1600 / 2000 Loss: 0.6011970639228821
Epoch: 1700 / 2000 Loss: 0.6690800786018372
Epoch: 1800 / 2000 Loss: 1.256234884262085
Epoch: 1900 / 2000 Loss: 1.3986696004867554
Epoch: 2000 / 2000 Loss: 0.7965167760848999


In [87]:
outs = myNet(torch.tensor(X_test.values).float())
outs = torch.argmax(outs, dim=1)
outs.shape


torch.Size([28000])

In [93]:
data = {
    'ImageId' : torch.arange(1, outs.shape[0]+1),
    'Label' : outs
}
df = pd.DataFrame(data)

In [94]:
df.to_csv("output_kan.csv", index=False)